In [2]:
from Util import get_openai_api_key
OPEN_API_KEY=get_openai_api_key()

In [3]:
llm_config={
    "config_list":{
        "model":"gpt-3.5-turbo",
        "api_key":OPEN_API_KEY
    }
}

In [5]:
import pandas as pd
dataset=pd.read_csv("sample_sales_data.csv")
newdata= dataset.to_string(index=False)

In [6]:
newdata

'region product  sales  quantity       date\n North  Laptop  15000         3 2025-07-01\n South  Tablet  12000         5 2025-07-01\n  East   Phone   9000         2 2025-07-02\n  West  Laptop  20000         4 2025-07-02\n South   Phone  11000         6 2025-07-03\n  East  Tablet   7000         3 2025-07-03\n North  Laptop  18000         5 2025-07-04\n  West   Phone   9500         1 2025-07-04\n South  Tablet  13000         4 2025-07-05\n  East  Laptop  17000         7 2025-07-05'

In [7]:
from autogen import UserProxyAgent,AssistantAgent,GroupChat,GroupChatManager

user_proxy=UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    code_execution_config={
        "use_docker":False
    }
)

In [9]:
validator_agent=AssistantAgent(
    name="validator_agent",
    llm_config=llm_config,
    system_message=f""" 
    You are an validator system assistant,
    Read the data which is already extracted from the newdata variable, and Show only rows where sales > 10,000 and region = 'South'.
    here is the dataset,
    {newdata}
    """
)

In [13]:
csv_data_converter=AssistantAgent(
    name="csv_data_converter",
    llm_config=llm_config,
    system_message=""" 
    you are an csv converter agent, based on the filtered results from validator agent.
    save the desired output results to the final_verdict.csv file
    You will receive a filtered dataset (as text table or CSV string).
    Task:
    - Convert it into pandas DataFrame.
    - Generate Python code to save it into 'verified_emails.csv'.
    Output only Python code inside a code block.
    """
)

In [14]:
group_chat=GroupChat(
    agents=[
        user_proxy,
        validator_agent,
        csv_data_converter
    ],
    messages=[],
    max_round=6
)

manager=GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config
)

In [15]:
response= user_proxy.initiate_chat(
    manager,
    message=f"""Can you read the dataset {newdata} and display the filtered results and convert it into an csv format file"""
)

user_proxy (to chat_manager):

Can you read the dataset region product  sales  quantity       date
 North  Laptop  15000         3 2025-07-01
 South  Tablet  12000         5 2025-07-01
  East   Phone   9000         2 2025-07-02
  West  Laptop  20000         4 2025-07-02
 South   Phone  11000         6 2025-07-03
  East  Tablet   7000         3 2025-07-03
 North  Laptop  18000         5 2025-07-04
  West   Phone   9500         1 2025-07-04
 South  Tablet  13000         4 2025-07-05
  East  Laptop  17000         7 2025-07-05 and display the filtered results and convert it into an csv format file

--------------------------------------------------------------------------------

Next speaker: validator_agent

validator_agent (to chat_manager):

Sure! Here are the rows where sales > 10,000 and region = 'South':

```
region product  sales  quantity       date
 South  Tablet  12000         5 2025-07-01
 South   Phone  11000         6 2025-07-03
 South  Tablet  13000         4 2025-07-05
```



In [16]:
import pandas as pd

# Create a DataFrame with the filtered data
data = {
    'region': ['South', 'South', 'South'],
    'product': ['Tablet', 'Phone', 'Tablet'],
    'sales': [12000, 11000, 13000],
    'quantity': [5, 6, 4],
    'date': ['2025-07-01', '2025-07-03', '2025-07-05']
}

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('final_verdict.csv', index=False)